In [1]:
import snscrape

def scrape_twitter_data(keyword):
    tweets = []
    for tweet in snscrape.modules.twitter.TwitterSearchScraper(keyword).get_items():
        tweets.append({
            'date': tweet.date,
            'id': tweet.id,
            'url': tweet.url,
            'content': tweet.text,
            'user': tweet.username,
            'reply_count': tweet.reply_count,
            'retweet_count': tweet.retweet_count,
            'language': tweet.language,
            'source': tweet.source,
            'like_count': tweet.like_count
        })
    return tweets

In [2]:
import pandas as pd
from pymongo import MongoClient

# Create a MongoDB client and a database
client = MongoClient()
db = client.twitter_data

def store_data_in_mongodb(keyword, tweets):
    # Create a collection for the keyword
    collection = db[keyword]
    # Insert the tweets into the collection
    collection.insert_many(tweets)

def get_data_from_mongodb(keyword):
    collection = db[keyword]
    return list(collection.find())

In [3]:
import streamlit as st

def create_gui():
    st.title("Twitter Data Scraper")
    keyword = st.text_input("Enter keyword or hashtag:")    
    date_range = st.date_input("Select date range:")
    tweet_count = st.number_input("Enter tweet count limit:")

    if st.button("Scrape Data"):
        tweets = scrape_twitter_data(keyword)
        # Filter tweets based on date range and tweet count limit
        tweets = [t for t in tweets if date_range[0] <= t['date'] <= date_range[1]][:tweet_count]
        df = pd.DataFrame(tweets)
        st.dataframe(df)
        if st.button("Upload to Database"):
            store_data_in_mongodb(keyword, tweets)
            st.success("Data uploaded to database!")
        if st.button("Download as CSV"):
            df.to_csv(f"{keyword}.csv")
            st.success("CSV downloaded!")
        if st.button("Download as JSON"):
            df.to_json(f"{keyword}.json")
            st.success("JSON downloaded!")

if __name__ == '__main__':
    create_gui()


2023-01-29 17:53:57.942 
  command:

    streamlit run C:\Users\91867\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
